In [17]:
import numpy as np
from evaldet.det.coco import evaluate_dataset
from evaldet.dist import iou_dist

import numba
from numba import types

from tqdm import tqdm

In [18]:
N_FRAMES = 1000
N_P_FRAME = 100

preds_bbox = np.random.rand(N_FRAMES * N_P_FRAME, 4).astype(np.float32)
gts_bbox = np.random.rand(N_FRAMES * N_P_FRAME, 4).astype(np.float32)
preds_conf = np.random.rand(N_FRAMES * N_P_FRAME).astype(np.float32)

fp = numba.typed.Dict.empty(
    key_type=numba.int32,
    value_type=types.Tuple((types.int32, types.int32))
)
fg = numba.typed.Dict.empty(
    key_type=numba.int32,
    value_type=types.Tuple((types.int32, types.int32))
)
ious = numba.typed.Dict.empty(
    key_type=numba.int32,
    value_type=numba.float32[:, ::1]
)

ious_np = {}
fp_np = {}
fg_np = {}

for f in tqdm(range(N_FRAMES)):
    inds = (f * N_P_FRAME, (f+1) * N_P_FRAME)
    fp[f] =inds
    fg[f] = inds
    ious[f] = 1 - iou_dist(preds_bbox[inds[0]:inds[1]], gts_bbox[inds[0]:inds[1]])

for f in tqdm(range(N_FRAMES)):
    inds = (f * N_P_FRAME, (f+1) * N_P_FRAME)
    fp_np[f] =inds
    fg_np[f] = inds
    ious_np[f] = 1 - iou_dist(preds_bbox[inds[0]:inds[1]], gts_bbox[inds[0]:inds[1]])

area_range = (0.5, 100)
iou_threshold = 0.5

100%|██████████| 1000/1000 [00:00<00:00, 2500.90it/s]


In [16]:
evaluate_dataset(
    preds_bbox, gts_bbox, ious, preds_conf, fp, fg, area_range, iou_threshold
)

(array([-1, 39, 89, ..., -1, -1, -1], dtype=int32),
 array([ True,  True, False, ...,  True,  True,  True]),
 array([False,  True,  True, ...,  True,  True,  True]),
 15347)

In [6]:
%%timeit
evaluate_dataset(
    preds_bbox, gts_bbox, ious, preds_conf, fp, fg, area_range, iou_threshold
)

12.8 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
calculate_pr_curve_np(
    preds_bbox, gts_bbox, ious_np, preds_conf, fp_np, fg_np, area_range, iou_threshold
)

array([[0.        , 0.5       , 0.6666667 , 0.75      , 0.8       ,
        0.6666667 , 0.71428573, 0.625     , 0.6666667 , 0.6       ,
        0.54545456, 0.5       , 0.46153846, 0.5       , 0.46666667,
        0.4375    , 0.4117647 , 0.44444445, 0.42105263, 0.4       ,
        0.3809524 , 0.36363637, 0.3478261 , 0.33333334, 0.36      ,
        0.3846154 , 0.37037036, 0.39285713, 0.37931034, 0.4       ,
        0.38709676, 0.40625   , 0.42424244, 0.4117647 , 0.4       ,
        0.3888889 , 0.3783784 , 0.36842105, 0.3846154 , 0.4       ,
        0.3902439 , 0.4047619 , 0.39534885, 0.38636363, 0.37777779,
        0.39130434, 0.40425533, 0.41666666, 0.40816328, 0.4       ,
        0.4117647 , 0.40384614, 0.3962264 , 0.4074074 , 0.4       ,
        0.39285713, 0.3859649 , 0.39655173, 0.40677965, 0.4       ,
        0.40983605, 0.41935483, 0.41269842, 0.40625   , 0.4       ,
        0.3939394 , 0.3880597 , 0.3970588 , 0.39130434, 0.3857143 ,
        0.3943662 , 0.4027778 , 0.41095892, 0.40

In [17]:
%%timeit
calculate_pr_curve_np(
    preds_bbox, gts_bbox, ious_np, preds_conf, fp_np, fg_np, area_range, iou_threshold
)

1.56 ms ± 89.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [1]:
import numpy as np
from evaldet import Detections

dets = Detections(
        image_ids=[0] * 3 + [1] * 3 + [2] * 2,
        bboxes=np.array(
            [
                [323.83, 104.06, 43.77, 35.43],
                [273.1, 88.77, 55.59, 24.32],
                [375.24, 80.43, 26.41, 22.24],
                [320.98, 105.24, 44.67, 35.71],
                [273.1, 88.88, 55.7, 24.52],
                [374.69, 80.78, 26.4, 22.23],
                [329.27, 96.65, 56.53, 32.45],
                [0.0, 356.7, 76.6, 122.67],
            ]
        ),
        classes=[2] * 8,
        confs=[0.9] * 8,
        class_names=["horse", "cat", "car"],
        image_names=["car1", "car2", "car3"],
    )

In [2]:
dets.to_coco("coco.json")

In [3]:
Detections.from_coco('coco.json')

In [1]:
import numpy as np
from evaldet.utils.confusion_matrix import confusion_matrix
from evaldet.det.coco import COCOMetrics
from evaldet.detections import Detections

/home/tadej/projects/EvalDeT/src/evaldet/det/coco.py:195: NumbaTypeSafetyWarning: unsafe cast from UniTuple(int64 x 2) to UniTuple(int32 x 2). Precision may be lost.
  preds_start, preds_end = img_ind_dict_preds.get(img, (0, 0))
/home/tadej/projects/EvalDeT/src/evaldet/det/coco.py:195: NumbaTypeSafetyWarning: unsafe cast from UniTuple(int64 x 2) to UniTuple(int32 x 2). Precision may be lost.
  preds_start, preds_end = img_ind_dict_preds.get(img, (0, 0))


In [2]:
hyp = Detections(
    image_ids=[0, 0, 0],
    bboxes=[np.array([0, 0, 1, 1]), np.array([1, 1, 1, 1]), np.array([2, 2, 1, 1])],
    classes=[0, 0, 1],
    class_names=("car", "truck"),
    image_names=("hello",),
    confs=[1,1,1]
    
)
gt = Detections(
    image_ids=[0, 0, 0],
    bboxes=[np.array([100, 100, 1, 1]), np.array([1, 1, 1, 1]), np.array([2, 2, 1, 1])],
    classes=[1, 0, 1],
    class_names=("car", "truck"),
    image_names=("hello",),
)

In [5]:
coco = COCOMetrics()
coco.confusion_matrix(gt, hyp, 0.5)

array([[1, 0, 1],
       [0, 1, 0],
       [0, 1, 0]], dtype=int32)

In [24]:
matching = np.array([1,2,-1])
gt_ignored = np.array([False, False, False])
hyp_classes = np.array([0,1,0])
gt_classes = np.array([1,1,0])
n_classes = 5

confusion_matrix(matching, gt_ignored, hyp_classes, gt_classes, n_classes)

array([[0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=int32)